In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings     
warnings.filterwarnings(action = "ignore")

import matplotlib.font_manager as fm  
font_name = fm.FontProperties(fname = "C:/Windows/Fonts/malgun.ttf").get_name()
plt.rc("font", family = font_name)

import matplotlib as mpl  
mpl.rcParams["axes.unicode_minus"] = False

import urllib.request as req
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
import urllib.parse
from bs4 import BeautifulSoup
import requests

## Selenium

### 1. 전체 프로젝트 크롤링

+ 전체 프로젝트(34,146개): 성사된 프로젝트(22,897개), 진행중인 프로젝트(571개), 공개 예정 프로젝트(60개) = 23,528
+ 34,146개여야 하는데 6156개만 크롤링됨--> '성사된 프로젝트' 수가 많아 이는 일부만 보여준 듯 함

In [29]:
from bs4 import BeautifulSoup
from selenium import webdriver

url = "https://tumblbug.com/discover?category=all"            
driver = webdriver.Chrome("C:/Users/admin/chromedriver")
driver.get(url)


In [10]:
# import time 

# while True:
#     last_height = driver.execute_script("return document.body.scrollHeight") 
    

#     driver.execute_script("window.scrollTo(document.body.scrollHeight/1.5, document.body.scrollHeight/1.5);")
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#     new_height = driver.execute_script("return document.body.scrollHeight")
    
# #     if new_height == last_height:        
# #         time.sleep(10)
# #         break

# html_source = driver.page_source 
# driver.close() 
# soup = BeautifulSoup(html_source, 'html.parser')

KeyboardInterrupt: 

In [ ]:
prev_height = driver.execute_script("return document.body.scrollHeight")
# 웹페이지 맨 아래까지 무한 스크롤
while True:
    # 스크롤을 화면 가장 아래로 내린다
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

    # 페이지 로딩 대기
    time.sleep(4)

    # 현재 문서 높이를 가져와서 저장
    curr_height = driver.execute_script("return document.body.scrollHeight")

    if (curr_height == prev_height):
        break
    else:
        prev_height = driver.execute_script("return document.body.scrollHeight")

In [ ]:
html_source = driver.page_source 
driver.close() 
soup = BeautifulSoup(html_source, 'html.parser')

In [139]:
soup

<html lang="ko-KR"><head>
<meta charset="utf-8"/>
<title>텀블벅 - 크리에이터를 위한 크라우드펀딩</title>
<meta content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1, user-scalable=no" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<link href="https://tumblbug-assets.imgix.net" rel="dns-prefetch"/>
<link href="https://tumblbug-pci2.imgix.net" rel="dns-prefetch"/>
<link href="https://tumblbug-psi.imgix.net" rel="dns-prefetch"/>
<link href="https://tumblbug-upi.imgix.net" rel="dns-prefetch"/>
<link href="https://www.google-analytics.com" rel="dns-prefetch"/>
<link href="https://developers.kakao.com" rel="dns-prefetch"/>
<link href="https://d2om2e6rfn032x.cloudfront.net" rel="dns-prefetch"/>
<link href="https://qysoaxc73e-dsn.algolia.net" rel="dns-prefetch"/>
<link href="https://stats.g.doubleclick.net" rel="dns-prefetch"/>
<link href="https://tumblbug.com/discover?category=all" rel="canonical"/>
<meta content="창의적인 시도를 함께 실현시키는 펀딩 커뮤니티 텀블벅에서 문화예술, 출판, 패션,

### 전체 프로젝트 크롤링 했을 경우 후처리

In [140]:
print(len(soup.find_all("dl")))

144


In [126]:
category1 = soup.find_all("dd", "project-sub-info")[0].a.get_text()

'향수 · 뷰티'

In [63]:
intro1 = soup.find_all("dd", "project-desc")[0].get_text()

6156


In [56]:
funding_amount1 = soup.find_all("span", "funding-amount")[0].get_text()

'26,942,399원'

In [48]:
percentage1 = soup.find_all("span", "percentage")[0].get_text()

'2694%'

In [53]:
rest_day1 = soup.find_all("span", "rest-day")[0].get_text()

'5일 남음'

In [135]:
success1 = soup.find_all("div", "ProjectCard__BackersInfo-opxl0a-7 fKywvA")[0].get_text()

'1,659명의 후원으로 펀딩 성공'

In [211]:
urll = soup.find_all("dt")[2].a["href"]

base = "https://tumblbug.com"
print(base + urll)

print(urllib.parse.urljoin(base, urll))

https://tumblbug.com/roomfriends
https://tumblbug.com/roomfriends


### 2. 데이터 추출

In [215]:
##### 데이터

# 전체 프로젝트: 성사된 프로젝트, 진행중인 프로젝트, 공개 예정 프로젝트(60개)

title = []
category = []
intro = []
funding_amount = []         # 진행중인 프로젝트에만 존재
percentage = []             # 진행중인 프로젝트에만 존재
rest_day = []               # 진행중인 프로젝트에만 존재
success = []               # 성사된 프로젝트에만 존재
url = [] 

for t in soup.find_all("dt"):
    title.append(t.get_text())

for c in soup.find_all("dd", "project-sub-info"):
    category.append(c.a.get_text())

for ir in soup.find_all("dd", "project-desc"):
    intro.append(ir.get_text())

for f in soup.find_all("span", "funding-amount"):
    funding_amount.append(f.get_text())
    
for p in soup.find_all("span", "percentage"):
    percentage.append(p.get_text())
    
for r in soup.find_all("span", "rest-day"):
    rest_day.append(r.get_text())
    
for s in soup.find_all("div", "ProjectCard__BackersInfo-opxl0a-7 fKywvA"):
    success.append(s.get_text())
    
for u in soup.find_all("dt"):
    urll = u.a["href"]
    base = "https://tumblbug.com"
    url.append(urllib.parse.urljoin(base, urll))

In [216]:
print("크롤링한 전체 프로젝트:", len(title), len(category), len(intro), len(url))
print("진행 중인 프로젝트인듯 함:", len(funding_amount), len(percentage), len(rest_day))
print("성사된 프로젝트가 너무 많으니 일부만 보여준듯 함:", len(success))
print("나머지 60개정도는 공개예정 프로젝트인듯 함")

#len(funding_amount) + len(success) + 60 = 6154

크롤링한 전체 프로젝트: 6156 6156 6156 6156
진행 중인 프로젝트인듯 함: 567 567 567
성사된 프로젝트가 너무 많으니 일부만 보여준듯 함: 5527
나머지 60개정도는 공개예정 프로젝트인듯 함


### 3. 데이터 프레임으로 저장

In [218]:
data = {"title":title,
        "category": category,
        "intro":intro,
        "url":url
       }

df = pd.DataFrame(data)
df

,title,category,intro,url
0,12년동안 5천개 제품 써보고 만든 속건조 진정 에센스,향수 · 뷰티,당신의 1억을 아껴드릴게요. 2년만에 완성한 속건조 잡는 저점도 고보습 에센스,https://tumblbug.com/gcbl
1,데뷔 못 하면 죽는 병 걸림 1차 공식 굿즈,출판,"[KW북스] 백덕수 작가님의 '데뷔 못 하면 죽는 병 걸림' 첫 번째 공식굿즈, 마...",https://tumblbug.com/testar_official_1st_goods
2,MZ세대를 위한 <한 권으로 끝내는 자취방 구하기>,출판,"자취방 구하기의 A to Z를 담은 가이드북과 메모지, 스티커, 엽서 문구세트를 소...",https://tumblbug.com/roomfriends
3,"웹툰, 일러스트를 위한 헤일로 후광과 문양 브러쉬!",웹툰 리소스,"헤일로 브러쉬, 키라키라 브러쉬, 문양 패턴 브러쉬, 프레임을 만나보세요!",https://tumblbug.com/gh03
4,문 시놉시스 : 웹소설 작가가 만든 하이퍼링크 시놉시스,출판,"웹소설은 물론 스토리텔링이 필요한 모든 곳에, 아마추어부터 프로작가까지 하이퍼링크 ...",https://tumblbug.com/moonsynopsis
...,...,...,...,...
6151,"올 여름, 더욱 빛날 ‘나’를 위한 은반지 SET",주얼리,"여름에 착용하기 좋은, 온전히 나를 표현하며 빛내줄 은반지 SET",https://tumblbug.com/why1
6152,잠들기 전의 설렘 : 문자 메세지 야광 뱃지,캐릭터 · 굿즈,아날로그 감성이 드는 야광 뱃지입니다. 밝을 때 빛을 머금고 있다가 어두워지면 빛나요.,https://tumblbug.com/analogmessage
6153,헬페미니스트 선언 - '그날' 이후의 페미니즘,출판,새롭게 등장한 페미니즘 주체 '헬페미'에 대한 이야기를 책으로 출간하고자 합니다.,https://tumblbug.com/hellfemi
6154,‘매직 더 개더링’ 작가의 신작 전략게임! ‘버니킹덤’,보드게임 · TRPG,게임의 역사를 바꾼 '매직더 개더링'개발자 리차드가필드의 신작 '버니킹덤',https://tumblbug.com/bunny


In [173]:
data_ing = {
        "funding_amount": funding_amount,
        "percentage":percentage,
        "rest_day":rest_day   
}
df_ing = pd.DataFrame(data_ing)
df_ing

,funding_amount,percentage,rest_day
0,"26,942,399원",2694%,5일 남음
1,"389,955,900원",3899%,16일 남음
2,"22,751,000원",4550%,6시간 남음
3,"3,189,000원",318%,18일 남음
4,"28,566,400원",5713%,23일 남음
...,...,...,...
562,0원,0%,34일 남음
563,0원,0%,16일 남음
564,0원,0%,2일 남음
565,0원,0%,48일 남음


In [174]:
data_success = {
        "success":success
}
df_success = pd.DataFrame(data_success)
df_success

,success
0,"1,659명의 후원으로 펀딩 성공"
1,"1,712명의 후원으로 펀딩 성공"
2,979명의 후원으로 펀딩 성공
3,635명의 후원으로 펀딩 성공
4,471명의 후원으로 펀딩 성공
...,...
5522,52명의 후원으로 펀딩 성공
5523,246명의 후원으로 펀딩 성공
5524,493명의 후원으로 펀딩 성공
5525,227명의 후원으로 펀딩 성공


### 4. 파일로 저장

In [219]:
##### CheckPoint 

df.to_csv("data/data.csv", encoding = "utf-8")
df_ing.to_csv("data/data_ing.csv", encoding = "utf-8")
df_success.to_csv("data/data_success.csv", encoding = "utf-8")

In [2]:
df1 = pd.read_csv("data/data.csv", index_col = 0)
df1

,title,category,intro,url
0,12년동안 5천개 제품 써보고 만든 속건조 진정 에센스,향수 · 뷰티,당신의 1억을 아껴드릴게요. 2년만에 완성한 속건조 잡는 저점도 고보습 에센스,https://tumblbug.com/gcbl
1,데뷔 못 하면 죽는 병 걸림 1차 공식 굿즈,출판,"[KW북스] 백덕수 작가님의 '데뷔 못 하면 죽는 병 걸림' 첫 번째 공식굿즈, 마...",https://tumblbug.com/testar_official_1st_goods
2,MZ세대를 위한 <한 권으로 끝내는 자취방 구하기>,출판,"자취방 구하기의 A to Z를 담은 가이드북과 메모지, 스티커, 엽서 문구세트를 소...",https://tumblbug.com/roomfriends
3,"웹툰, 일러스트를 위한 헤일로 후광과 문양 브러쉬!",웹툰 리소스,"헤일로 브러쉬, 키라키라 브러쉬, 문양 패턴 브러쉬, 프레임을 만나보세요!",https://tumblbug.com/gh03
4,문 시놉시스 : 웹소설 작가가 만든 하이퍼링크 시놉시스,출판,"웹소설은 물론 스토리텔링이 필요한 모든 곳에, 아마추어부터 프로작가까지 하이퍼링크 ...",https://tumblbug.com/moonsynopsis
...,...,...,...,...
6151,"올 여름, 더욱 빛날 ‘나’를 위한 은반지 SET",주얼리,"여름에 착용하기 좋은, 온전히 나를 표현하며 빛내줄 은반지 SET",https://tumblbug.com/why1
6152,잠들기 전의 설렘 : 문자 메세지 야광 뱃지,캐릭터 · 굿즈,아날로그 감성이 드는 야광 뱃지입니다. 밝을 때 빛을 머금고 있다가 어두워지면 빛나요.,https://tumblbug.com/analogmessage
6153,헬페미니스트 선언 - '그날' 이후의 페미니즘,출판,새롭게 등장한 페미니즘 주체 '헬페미'에 대한 이야기를 책으로 출간하고자 합니다.,https://tumblbug.com/hellfemi
6154,‘매직 더 개더링’ 작가의 신작 전략게임! ‘버니킹덤’,보드게임 · TRPG,게임의 역사를 바꾼 '매직더 개더링'개발자 리차드가필드의 신작 '버니킹덤',https://tumblbug.com/bunny


In [3]:
df3 = pd.read_csv("data/11400ea.csv", index_col = 0)
df3

,title,category,intro,url
0,웹툰과 일러스트를 위한 꽃브러쉬,웹툰 리소스,웹툰과 일러스트에 사용할수있는 꽃소재입니다.,https://tumblbug.com/didrkrk
1,오디오웹툰으로 만나는 <고래별_경성의 인어공주> 시즌2,웹툰 · 만화,많은 사랑을 받았던 네이버웹툰 <고래별> 오디오웹툰 펀딩이 시즌2로 돌아옵니다!,https://tumblbug.com/thewhalestar2
2,꼬마마법사 레미 공식 한정판 굿즈,영화 · 비디오,방영 20주년 기념 '꼬마마법사 레미' 공식 한정판 굿즈를 소개합니다!,https://tumblbug.com/doremi
3,"한국영화 스태프들이 묻고 답하다, <영화가 뭐라고>",출판,각자 다른 분야에서 일하는 한국영화 스태프 32명의 업무와 고민을 담은 인터뷰집,https://tumblbug.com/secondary_project
4,만화가의 금융에세이 <자산소득으로 생활하기 프로젝트>,출판,1억1천400만원을 모으겠어! 노동탈출 금융에세이 <자산소득으로 생활하기 프로젝트>...,https://tumblbug.com/financial_lifebook
...,...,...,...,...
11371,메카피아의 한국 전통 자개 문양 피젯 스피너,홈 · 리빙,전세계에 하나밖에 없는 한국적인 디자인의 자개 문양 메탈 피젯 스피너 프로젝트,https://tumblbug.com/jagefidget
11372,카프카 <변신> 초판본과 독서카드,출판,읽고 쓰는 기쁨을 위해 책방 카프카의밤이 마련한 굿즈,https://tumblbug.com/goodnight_kafka
11373,낯선 첫발을 내딛는 이방인을 위한 <연남동 작은 방>,출판,"낯선 첫발을 내딛는 이들을 위한 에세이. 쓸쓸한 안식의, 1인분의 방 <연남동 작은 방>",https://tumblbug.com/mysmallplace
11374,함께 행복하개! 유기견 팔찌,캐릭터 · 굿즈,사람과 동물이 함께 행복한 세상을 만들어요~ 수익금은 동물자유연대에 기부할 예정입니다.,https://tumblbug.com/deforest
